INSTALLATION OF DEPENDCIES

In [ ]:
# Install necessary packages
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tensorflow
!pip install tensorflow-addons



In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ccdmu6wf
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ccdmu6wf
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=7acf30e7a2592d04b38ff93cef674b6ad0d8d78acc79286affe1ef5e3adae902
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdiytp_e/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


LSTM MODEL

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
from keras.optimizers import Adam

# Read data from CSV file
data = pd.read_csv("MAT_Results.csv")

# Extracting X and y
X = data.iloc[:, 1:97].values  # Assuming columns 1 to 96 are features
y = data[['Grade_Level', 'Scores']].values  # Using columns 'Grade_Level' and 'Scores' as targets

# Ensure X is 3D: (num_samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))  # Reshape for LSTM input if needed

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=50, return_sequences=False, recurrent_dropout=0.1), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.1))
model.add(Dense(50, activation="relu"))
model.add(Dense(2))  # Output layer for grade and score

# Compile model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.1)

# Evaluate model
scores = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {scores[1]*100:.2f}%")

# Make predictions
predictions = model.predict(X_test)

# Post-processing predictions to get grade and score
predicted_grade = predictions[:, 0]
predicted_score = predictions[:, 1]

# Create a DataFrame to store predictions
predictions_df = pd.DataFrame({
    'Predicted_Grade': predicted_grade,
    'Predicted_Score': predicted_score,
})

# Save predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

# Save the model
model.save('lstm_model.h5')


Epoch 1/5
28/28 [==============================] - 6s 29ms/step - loss: 127.8283 - accuracy: 0.6678 - val_loss: 117.9555 - val_accuracy: 0.6224
Epoch 2/5
28/28 [==============================] - 0s 8ms/step - loss: 91.8704 - accuracy: 0.6678 - val_loss: 84.6277 - val_accuracy: 0.6224
Epoch 3/5
28/28 [==============================] - 0s 8ms/step - loss: 71.1539 - accuracy: 0.6633 - val_loss: 74.7484 - val_accuracy: 0.6837
Epoch 4/5
28/28 [==============================] - 0s 8ms/step - loss: 67.3763 - accuracy: 0.5340 - val_loss: 73.1023 - val_accuracy: 0.4388
Epoch 5/5
8/8 [==============================] - 0s 3ms/step - loss: 64.3409 - accuracy: 0.4122
Model Accuracy: 41.22%
8/8 [==============================] - 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


LOGISTIC REGRESSION

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import plotly.express as px

# Load data
data_df = pd.read_csv('MAT_fTrain.csv')

X_features = ['AWL', 'TTR', '-LRB-', '-RRB-', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD', 'COMMA', 'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DPAR', 'DT', 'DWNT', 'EMPH', 'EX', 'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'LS', 'NEMD', 'NN', 'NOMZ', 'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED', 'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[CONT]', '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRESP]', '[PRIV]', '[PROD]', '[PUBV]', '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[STPR]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]', '_NN']

# X_features= data_df.drop(['Grade_Level'], axis=1)
print(f"Number of features in X_features: {len(X_features)}")
X = data_df.drop(['Grade_Level'], axis=1)  # Features
y = data_df['Grade_Level']  # Target variable for grade level
print(f"Columns in X: {X.columns.tolist()}")


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Model and fit
log_reg = LogisticRegression(max_iter=10000, multi_class='multinomial', solver='lbfgs')
log_reg_model = log_reg.fit(X_train, y_train)

# Predictions
y_pred = log_reg_model.predict(X_test)
y_pred_probabilities = log_reg_model.predict_proba(X_test)

# Classification report
classification_report = metrics.classification_report(y_test, y_pred)
print(classification_report)
# Save classification report to a text file
with open('LogisticRegression_classification_report.txt', 'w') as f:
    f.write(classification_report)

# Confusion matrix
confusion = metrics.confusion_matrix(y_test, y_pred)
print(confusion)
# Accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

# Feature importance
feature_importance = log_reg_model.coef_
feature_importance_df = pd.DataFrame(feature_importance, columns=X_features)
feature_importance_df['Grade'] = log_reg_model.classes_
feature_importance_df = feature_importance_df.set_index('Grade')
feature_importance_df.to_csv('LogisticRegression_feature_importance_multiclass.csv')

# ROC Curve and AUC
fig = px.area(
    title='ROC Curve',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)





Number of features in X_features: 91
Columns in X: ['AWL', 'TTR', '-LRB-', '-RRB-', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD', 'COMMA', 'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DPAR', 'DT', 'DWNT', 'EMPH', 'EX', 'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'LS', 'NEMD', 'NN', 'NOMZ', 'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED', 'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[CONT]', '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRESP]', '[PRIV]', '[PROD]', '[PUBV]', '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[STPR]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]', '_NN']
              precision    recall  f1-score   support

           7       0.86      0.85      0.85       303
           8       0.87      0.81      0.84       716
          10       0.90      0.93      0.91   

SAVE THE MODEL

In [ ]:
!pip install joblib


In [ ]:
import joblib
joblib.dump(log_reg_model, 'logistic_regression_model.pkl')


CNN MODEL USING CLASSIFIER

In [ ]:
!pip install tensorflow


CNN MODEL - FINAL

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from sklearn.inspection import permutation_importance
from sklearn.base import BaseEstimator, ClassifierMixin
import plotly.express as px

# Load data
data_df = pd.read_csv('MAT_fTrain.csv')

# Features and target
X_features = ['AWL', 'TTR', '-LRB-', '-RRB-', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD', 'COMMA', 'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DPAR', 'DT', 'DWNT', 'EMPH', 'EX', 'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'LS', 'NEMD', 'NN', 'NOMZ', 'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED', 'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[CONT]', '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRESP]', '[PRIV]', '[PROD]', '[PUBV]', '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[STPR]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]', '_NN']

print(f"Number of features in X_features: {len(X_features)}")
X = data_df[X_features]
y = data_df['Grade_Level']
print(f"Columns in X: {X.columns.tolist()}")

# Encoding target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data to fit the CNN model
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_categorical, test_size=0.20, random_state=42)

# Define a wrapper class for Keras model
class KerasEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y, epochs=30, batch_size=32, verbose=0)
        return self

    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=1)

    def score(self, X, y):
        return metrics.accuracy_score(y, self.predict(X))

# Create the model
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create an instance of the wrapper class
keras_estimator = KerasEstimator(create_cnn_model((X_reshaped.shape[1], 1), len(label_encoder.classes_)))

# Fit the model on the training data
keras_estimator.fit(X_train, y_train)

# Initialize a DataFrame to store feature importances
feature_importance_df = pd.DataFrame(columns=['Grade_Level', 'Feature', 'Importance_Mean'])

# Calculate permutation importance for each grade level
for grade_level in label_encoder.classes_:
    mask = (y == grade_level)
    X_grade_level = X_scaled[mask]
    y_grade_level = y_encoded[mask]

    result = permutation_importance(keras_estimator, X_grade_level, y_grade_level, n_repeats=10, random_state=42)

    # Append results to the DataFrame
    temp_df = pd.DataFrame({
        'Grade_Level': grade_level,
        'Feature': X.columns,
        'Importance_Mean': result.importances_mean
    })
    feature_importance_df = pd.concat([feature_importance_df, temp_df], ignore_index=True)

# Save the DataFrame to a CSV file
feature_importance_df.to_csv('feature_importance_for_Grade_Level.csv', index=False)

# Print feature importance DataFrame
print(feature_importance_df)

# Predict on the test set
y_test_pred = keras_estimator.predict(X_test)
y_test_true = np.argmax(y_test, axis=1)

# Convert label_encoder.classes_ to a list of strings
target_names = [str(label) for label in label_encoder.classes_]

# Classification report
classification_report = metrics.classification_report(y_test_true, y_test_pred, target_names=target_names)
print("Classification Report:\n", classification_report)
with open('CNN_classification_report.txt', 'w') as f:
    f.write(classification_report)

# Confusion matrix
confusion_matrix = metrics.confusion_matrix(y_test_true, y_test_pred)
print("Confusion Matrix:\n", confusion_matrix)

# Accuracy
accuracy = metrics.accuracy_score(y_test_true, y_test_pred)
print('Accuracy:', accuracy)




Number of features in X_features: 91
Columns in X: ['AWL', 'TTR', '-LRB-', '-RRB-', 'AMP', 'ANDC', 'CAUS', 'CC', 'CD', 'COMMA', 'CONC', 'COND', 'CONJ', 'DEMO', 'DEMP', 'DPAR', 'DT', 'DWNT', 'EMPH', 'EX', 'FPP1', 'FW', 'GER', 'HDG', 'IN', 'INPR', 'JJ', 'LS', 'NEMD', 'NN', 'NOMZ', 'OSUB', 'PDT', 'PHC', 'PIN', 'PIT', 'PLACE', 'POMD', 'POS', 'PRED', 'PRMD', 'PRP', 'QUAN', 'QUPR', 'RB', 'RP', 'SPP2', 'SYM', 'SYNE', 'THAC', 'THVC', 'TIME', 'TO', 'TOBJ', 'TPP3', 'TSUB', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VPRT', 'WDT', 'WP', 'WPS', 'XX0', '[BEMA]', '[BYPA]', '[CONT]', '[PASS]', '[PASTP]', '[PEAS]', '[PIRE]', '[PRESP]', '[PRIV]', '[PROD]', '[PUBV]', '[SERE]', '[SMP]', '[SPAU]', '[SPIN]', '[STPR]', '[SUAV]', '[THATD]', '[WHCL]', '[WHOBJ]', '[WHQU]', '[WHSUB]', '[WZPAST]', '[WZPRES]', '_NN']
245/245 [==============================] - 1s 6ms/step
    Grade_Level   Feature  Importance_Mean
0             7       AWL         0.000064
1             7       TTR         0.003760
2             7     -LRB-

In [ ]:
# Fit the model on the training data
keras_estimator.fit(X_train, y_train)

# Save the trained model
keras_estimator.model.save('keras_cnn_model.h5')

# Load the model later using:
# loaded_model = tf.keras.models.load_model('keras_cnn_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
# Convert label_encoder.classes_ to a list of strings
target_names = [str(label) for label in label_encoder.classes_]

# Classification report
classification_report = metrics.classification_report(y_test_true, y_test_pred, target_names=target_names)
print("Classification Report:\n", classification_report)
with open('CNN_classification_report.txt', 'w') as f:
    f.write(classification_report)

# Confusion matrix
confusion_matrix = metrics.confusion_matrix(y_test_true, y_test_pred)
print("Confusion Matrix:\n", confusion_matrix)

# Accuracy
accuracy = metrics.accuracy_score(y_test_true, y_test_pred)
print('Accuracy:', accuracy)

Classification Report:
               precision    recall  f1-score   support

           7       0.94      0.88      0.91        33
           8       0.81      0.88      0.84        76
          10       0.90      0.87      0.88       136

    accuracy                           0.87       245
   macro avg       0.88      0.88      0.88       245
weighted avg       0.88      0.87      0.87       245

Confusion Matrix:
 [[ 29   0   4]
 [  0  67   9]
 [  2  16 118]]
Accuracy: 0.8734693877551021


# New Section